In [1]:
import requests
import nacl.signing
import base64
import os
import hashlib
import random

In [2]:
def csrf_token(nbytes=32):
    tok = os.urandom(nbytes)
    return base64.urlsafe_b64encode(tok).rstrip(b'=').decode('ascii')

def code_challenge(length=128):
    choices = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-._~'
    code_verifier = "".join(random.choice(choices) for _ in range(length))
    return base64.urlsafe_b64encode(hashlib.sha256(code_verifier).digest())

In [3]:
# retrieve a list of instances
inst_doc_url = 'https://static.eduvpn.nl/instances.json'
inst_doc_sig_url = 'https://static.eduvpn.nl/instances.json.sig'
inst_doc = requests.get(inst_doc_url)
inst_doc_sig = requests.get(inst_doc_sig_url)
instances = [i['base_uri'] for i in inst_doc.json()['instances']]
instances

[u'https://nl.eduvpn.org/',
 u'https://esciencecenter.eduvpn.nl/',
 u'https://ru.eduvpn.nl/',
 u'https://surf.eduvpn.nl/',
 u'https://ut.eduvpn.nl/',
 u'https://demo.eduvpn.nl/']

In [4]:
#verify_key = nacl.signing.VerifyKey(key, encoder=nacl.encoding.Base64Encoder)
#verify_key.verify(smessage=inst_doc.content, signature=inst_doc_sig.content)

In [5]:
instance = 'https://demo.eduvpn.nl/'

In [6]:
info = requests.get(instance + '/info.json').json()
urls = info['api']['http://eduvpn.org/api#2']
authorization_endpoint = urls['authorization_endpoint']

In [7]:
params = {
    "client_id": "gijs",   #  the ID that was registered, see below;
    "redirect_uri": "http://localhost",   # the URL that was registered, see below;
    "response_type": "code",  # always code;
    "scope": "config",  # this is always config;
    "state": csrf_token(),  # a cryptographically secure random string, to avoid CSRF;
    "code_challenge_method": "S256",  # always S256;
    "code_challenge": code_challenge().replace('=', '') # the code challenge (see RFC 7636).
}
print(authorization_endpoint + "?" + "&".join([k + "=" + v for k, v in params.items()]))

https://demo.eduvpn.nl/portal/_oauth/authorize?code_challenge=V8s5Jy0rlWRglXK73TcP8O0iqvQkvNhR4AHPJs5Qkgc&client_id=gijs&state=QOGoj6gCfgVOjEyLDJ2C9v9aODYBnlx175M5vVV3s2o&redirect_uri=http://localhost&response_type=code&code_challenge_method=S256&scope=config
